In [1]:
import requests
import numpy as np
import vlc

def get_cctv_url(lat, lng):
    # CCTV 탐색 범위 지정을 위해 임의로 ±1 만큼 가감
    minX = str(lng-1)
    maxX = str(lng+1)
    minY = str(lat-1)
    maxY = str(lat+1)

    # 개인key 입력
    api_call = 'https://openapi.its.go.kr:9443/cctvInfo?' \
               'apiKey=94720f90c8be454683ebf390f2904e78' \
               '&type=ex&cctvType=2' \
               '&minX=' + minX + \
               '&maxX=' + maxX + \
               '&minY=' + minY + \
               '&maxY=' + maxY + \
               '&getType=json'

    w_dataset = requests.get(api_call).json()
    cctv_data = w_dataset['response']['data']

    coordx_list = []
    for index, data in enumerate(cctv_data):
        xy_couple = (float(cctv_data[index]['coordy']),float(cctv_data[index]['coordx']))
        coordx_list.append(xy_couple)

    # 입력한 위경도 좌표에서 가장 가까운 위치에 있는 CCTV를 찾는 과정
    coordx_list = np.array(coordx_list)
    leftbottom = np.array((lat, lng))
    distances = np.linalg.norm(coordx_list - leftbottom, axis=1)
    min_index = np.argmin(distances)

    return cctv_data[min_index]


cctv_data = get_cctv_url(36.58629, 128.186793)

print('CCTV명:', cctv_data['cctvname']) # 가장 가까운 CCTV명
print('CCTV 영상 URL:', cctv_data['cctvurl']) # 가장 가까운 CCTV 영상 URL

CCTV명: [중부내륙선] 나한교
CCTV 영상 URL: http://cctvsec.ktict.co.kr/3995/JKYW8SJfet3w0An+BhkoEZiYEAGJOD7GNlI3aKLE0kOcB5BWxH68ueB+TTLL25vISfsyR8X99EFj4QRB6Hdfcw==


In [2]:
import requests
import vlc

def get_all_cctv_urls(lat, lng):
    # CCTV 탐색 범위 지정을 위해 ±1 만큼 가감
    minX = str(lng - 1)
    maxX = str(lng + 1)
    minY = str(lat - 1)
    maxY = str(lat + 1)

    # 개인 API 키 입력
    api_call = 'https://openapi.its.go.kr:9443/cctvInfo?' \
               'apiKey=94720f90c8be454683ebf390f2904e78' \
               '&type=ex&cctvType=1' \
               '&minX=' + minX + \
               '&maxX=' + maxX + \
               '&minY=' + minY + \
               '&maxY=' + maxY + \
               '&getType=json'

    # API 호출 및 JSON 데이터 가져오기
    response = requests.get(api_call)
    if response.status_code != 200:
        print("API 호출 실패:", response.status_code)
        return []
    
    try:
        w_dataset = response.json()
        cctv_data = w_dataset.get('response', {}).get('data', [])
        return cctv_data
    except Exception as e:
        print("JSON 파싱 중 오류 발생:", e)
        return []

def search_cctv_by_name(lat, lng, search_keyword):
    # 모든 CCTV 데이터를 가져오는 함수 호출
    cctv_list = get_all_cctv_urls(lat, lng)

    # 검색 키워드가 포함된 CCTV를 필터링
    filtered_cctvs = [cctv for cctv in cctv_list if search_keyword in cctv['cctvname']]

    # 검색 결과 반환
    return filtered_cctvs

def stream_cctv_vlc(cctv_url):
    # VLC 플레이어 초기화
    player = vlc.MediaPlayer(cctv_url)
    player.play()

    print("CCTV 스트림이 시작되었습니다. 종료하려면 'Ctrl + C'를 누르세요.")
    try:
        while True:
            pass  # 스트리밍 계속 실행
    except KeyboardInterrupt:
        print("스트림을 종료합니다.")
        player.stop()

# 특정 지역 위도와 경도
lat = 36.58629
lng = 128.186793

# 검색할 CCTV 이름 키워드
search_keyword = input("검색할 CCTV 이름 키워드를 입력하세요: ")

# 이름으로 CCTV 검색
result = search_cctv_by_name(lat, lng, search_keyword)

# 결과 출력 및 스트리밍 선택
if result:
    print(f"'{search_keyword}'이(가) 포함된 CCTV 목록:")
    for i, cctv in enumerate(result):
        print(f"{i+1}. CCTV명: {cctv['cctvname']} - URL: {cctv['cctvurl']}")
    
    # 사용자로부터 재생할 CCTV 선택
    while True:
        try:
            choice = int(input(f"재생할 CCTV를 선택하세요 (1~{len(result)}): ")) - 1
            if 0 <= choice < len(result):
                cctv_url = result[choice]['cctvurl']
                break
            else:
                print("잘못된 번호입니다. 다시 선택하세요.")
        except ValueError:
            print("숫자를 입력하세요.")

else:
    print(f"'{search_keyword}'에 해당하는 CCTV를 찾을 수 없습니다.")


'전주'이(가) 포함된 CCTV 목록:
1. CCTV명: [순천완주선] 동전주 - URL: http://cctvsec.ktict.co.kr/3199/+dI1YYFU739JAhP3OMCZyiLJROq4imCy9fzJ1trYzCLPDsYtRMS9tJTE3dXqnD4h8uJOYQnM5TWEslQ51vv9rqPFNEM/fLawC32L14X36gI=


In [3]:
#-----------------------------------------------------------------------------------------------------------------------------------#

In [3]:
import requests
import cv2
import torch
from yolov5 import detect

def detect_cars_yolov5(frame, model, confidence_threshold=0.5):
    # YOLOv5 모델로 자동차 탐지
    results = model(frame)  # 프레임 입력
    detections = results.pandas().xyxy[0]  # 결과를 DataFrame 형태로 반환
    detections = detections[detections['confidence'] > confidence_threshold]  # 신뢰도 필터링
    car_detections = detections[detections['name'] == 'car']  # 'car' 클래스만 필터링
    return car_detections

def stream_and_detect_cars(cctv_url):
    # YOLOv5 모델 로드
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # yolov5s: 가장 가벼운 모델
    print("YOLOv5 모델이 로드되었습니다.")

    # CCTV 스트리밍 시작
    cap = cv2.VideoCapture(cctv_url)
    if not cap.isOpened():
        print("CCTV 스트림을 열 수 없습니다.")
        return

    print("CCTV 스트림이 시작되었습니다. 종료하려면 'q'를 누르세요.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("프레임을 읽을 수 없습니다.")
            break

        # 자동차 탐지
        detections = detect_cars_yolov5(frame, model)

        # 탐지된 자동차 표시
        for _, row in detections.iterrows():
            x1, y1, x2, y2, confidence = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax']), row['confidence']
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Car: {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 자동차 수 출력
        car_count = len(detections)
        cv2.putText(frame, f"Car count: {car_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # 화면에 프레임 출력
        cv2.imshow("CCTV Stream", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
stream_and_detect_cars('http://cctvsec.ktict.co.kr/3199/+dI1YYFU739JAhP3OMCZyiLJROq4imCy9fzJ1trYzCLPDsYtRMS9tJTE3dXqnD4hySR9bJ3QZPLGR/hWjbNLOgYfb+reTgZfaVFgqylP7Sw=')

Using cache found in C:\Users\SANGMIN/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-12-18 Python-3.10.13 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


YOLOv5 모델이 로드되었습니다.
CCTV 스트림이 시작되었습니다. 종료하려면 'q'를 누르세요.


In [6]:
import requests
import cv2
import torch
from yolov5 import detect

def detect_cars_yolov5(frame, model, confidence_threshold=0.5):
    # YOLOv5 모델로 자동차 탐지
    results = model(frame)  # 프레임 입력
    detections = results.pandas().xyxy[0]  # 결과를 DataFrame 형태로 반환
    detections = detections[detections['confidence'] > confidence_threshold]  # 신뢰도 필터링
    # 'car'와 'truck' 클래스를 필터링
    car_detections = detections[detections['name'].isin(['car', 'truck'])]  
    return car_detections

def stream_and_detect_cars(cctv_url):
    # YOLOv5 모델 로드
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # yolov5s: 가장 가벼운 모델
    print("YOLOv5 모델이 로드되었습니다.")

    # CCTV 스트리밍 시작
    cap = cv2.VideoCapture(cctv_url)
    if not cap.isOpened():
        print("CCTV 스트림을 열 수 없습니다.")
        return

    print("CCTV 스트림이 시작되었습니다. 종료하려면 'q'를 누르세요.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("프레임을 읽을 수 없습니다.")
            break

        # 자동차 탐지
        detections = detect_cars_yolov5(frame, model)

        # 탐지된 자동차 표시
        for _, row in detections.iterrows():
            x1, y1, x2, y2, confidence = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax']), row['confidence']
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Car: {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 자동차 수 출력
        car_count = len(detections)
        cv2.putText(frame, f"Car count: {car_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # 화면에 프레임 출력
        cv2.imshow("CCTV Stream", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def get_all_cctv_urls(lat, lng):
    # CCTV 탐색 범위 지정을 위해 ±1 만큼 가감
    minX = str(lng - 1)
    maxX = str(lng + 1)
    minY = str(lat - 1)
    maxY = str(lat + 1)

    # 개인 API 키 입력
    api_call = 'https://openapi.its.go.kr:9443/cctvInfo?' \
               'apiKey=94720f90c8be454683ebf390f2904e78' \
               '&type=ex&cctvType=1' \
               '&minX=' + minX + \
               '&maxX=' + maxX + \
               '&minY=' + minY + \
               '&maxY=' + maxY + \
               '&getType=json'

    # API 호출 및 JSON 데이터 가져오기
    response = requests.get(api_call)
    if response.status_code != 200:
        print("API 호출 실패:", response.status_code)
        return []
    
    try:
        w_dataset = response.json()
        cctv_data = w_dataset.get('response', {}).get('data', [])
        return cctv_data
    except Exception as e:
        print("JSON 파싱 중 오류 발생:", e)
        return []

def search_cctv_by_name(lat, lng, search_keyword):
    # 모든 CCTV 데이터를 가져오는 함수 호출
    cctv_list = get_all_cctv_urls(lat, lng)

    # 검색 키워드가 포함된 CCTV를 필터링
    filtered_cctvs = [cctv for cctv in cctv_list if search_keyword in cctv['cctvname']]

    # 검색 결과 반환
    return filtered_cctvs

# 특정 지역 위도와 경도
lat = 36.58629
lng = 128.186793

# 검색할 CCTV 이름 키워드
search_keyword = input("검색할 CCTV 이름 키워드를 입력하세요: ")

# 이름으로 CCTV 검색
result = search_cctv_by_name(lat, lng, search_keyword)

# 결과 출력 및 스트리밍 선택
if result:
    print(f"'{search_keyword}'이(가) 포함된 CCTV 목록:")
    for i, cctv in enumerate(result):
        print(f"{i+1}. CCTV명: {cctv['cctvname']} - URL: {cctv['cctvurl']}")
    
    # 사용자로부터 재생할 CCTV 선택
    while True:
        try:
            choice = int(input(f"재생할 CCTV를 선택하세요 (1~{len(result)}): ")) - 1
            if 0 <= choice < len(result):
                cctv_url = result[choice]['cctvurl']
                break
            else:
                print("잘못된 번호입니다. 다시 선택하세요.")
        except ValueError:
            print("숫자를 입력하세요.")

    # 선택된 URL로 자동차 탐지 실행
    stream_and_detect_cars(cctv_url)
else:
    print(f"'{search_keyword}'에 해당하는 CCTV를 찾을 수 없습니다.")


'전주'이(가) 포함된 CCTV 목록:
1. CCTV명: [순천완주선] 동전주 - URL: http://cctvsec.ktict.co.kr/3199/+dI1YYFU739JAhP3OMCZyiLJROq4imCy9fzJ1trYzCLPDsYtRMS9tJTE3dXqnD4hapqb7oaa4sgVxsPC9fY/tGx2nuyCeibC1Czx5vF8754=


Using cache found in C:\Users\SANGMIN/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-12-18 Python-3.10.13 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


YOLOv5 모델이 로드되었습니다.
CCTV 스트림이 시작되었습니다. 종료하려면 'q'를 누르세요.


In [5]:
import requests
import cv2
import torch
from yolov5 import detect

def detect_cars_yolov5(frame, model, confidence_threshold=0.5, iou_threshold=0.45):
    # Non-Maximum Suppression (NMS) 추가
    results = model(frame)
    detections = results.pandas().xyxy[0]
    detections = detections[detections['confidence'] > confidence_threshold]
    car_detections = detections[detections['name'].isin(['car', 'truck'])]
    
    # NMS 적용: 겹치는 박스 제거
    unique_detections = cv2.dnn.NMSBoxes(
        car_detections[['xmin', 'ymin', 'xmax', 'ymax']].values.tolist(), 
        car_detections['confidence'].tolist(), 
        confidence_threshold, 
        iou_threshold
    )
    return car_detections.iloc[unique_detections]

def stream_and_detect_cars(cctv_url):
    # YOLOv5 모델 로드
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # yolov5s: 가장 가벼운 모델
    print("YOLOv5 모델이 로드되었습니다.")

    # CCTV 스트리밍 시작
    cap = cv2.VideoCapture(cctv_url)
    if not cap.isOpened():
        print("CCTV 스트림을 열 수 없습니다.")
        return

    print("CCTV 스트림이 시작되었습니다. 종료하려면 'q'를 누르세요.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("프레임을 읽을 수 없습니다.")
            break

        # 자동차 탐지
        detections = detect_cars_yolov5(frame, model)

        # 탐지된 자동차 표시
        for _, row in detections.iterrows():
            x1, y1, x2, y2, confidence = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax']), row['confidence']
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Car: {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 자동차 수 출력
        car_count = len(detections)
        cv2.putText(frame, f"Car count: {car_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # 화면에 프레임 출력
        cv2.imshow("CCTV Stream", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def get_all_cctv_urls(lat, lng):
    # CCTV 탐색 범위 지정을 위해 ±1 만큼 가감
    minX = str(lng - 1)
    maxX = str(lng + 1)
    minY = str(lat - 1)
    maxY = str(lat + 1)

    # 개인 API 키 입력
    api_call = 'https://openapi.its.go.kr:9443/cctvInfo?' \
               'apiKey=94720f90c8be454683ebf390f2904e78' \
               '&type=ex&cctvType=1' \
               '&minX=' + minX + \
               '&maxX=' + maxX + \
               '&minY=' + minY + \
               '&maxY=' + maxY + \
               '&getType=json'

    # API 호출 및 JSON 데이터 가져오기
    response = requests.get(api_call)
    if response.status_code != 200:
        print("API 호출 실패:", response.status_code)
        return []
    
    try:
        w_dataset = response.json()
        cctv_data = w_dataset.get('response', {}).get('data', [])
        return cctv_data
    except Exception as e:
        print("JSON 파싱 중 오류 발생:", e)
        return []

def search_cctv_by_name(lat, lng, search_keyword):
    # 모든 CCTV 데이터를 가져오는 함수 호출
    cctv_list = get_all_cctv_urls(lat, lng)

    # 검색 키워드가 포함된 CCTV를 필터링
    filtered_cctvs = [cctv for cctv in cctv_list if search_keyword in cctv['cctvname']]

    # 검색 결과 반환
    return filtered_cctvs

# 특정 지역 위도와 경도
lat = 36.58629
lng = 128.186793

# 검색할 CCTV 이름 키워드
search_keyword = input("검색할 CCTV 이름 키워드를 입력하세요: ")

# 이름으로 CCTV 검색
result = search_cctv_by_name(lat, lng, search_keyword)

# 결과 출력 및 스트리밍 선택
if result:
    print(f"'{search_keyword}'이(가) 포함된 CCTV 목록:")
    for i, cctv in enumerate(result):
        print(f"{i+1}. CCTV명: {cctv['cctvname']} - URL: {cctv['cctvurl']}")
    
    # 사용자로부터 재생할 CCTV 선택
    while True:
        try:
            choice = int(input(f"재생할 CCTV를 선택하세요 (1~{len(result)}): ")) - 1
            if 0 <= choice < len(result):
                cctv_url = result[choice]['cctvurl']
                break
            else:
                print("잘못된 번호입니다. 다시 선택하세요.")
        except ValueError:
            print("숫자를 입력하세요.")

    # 선택된 URL로 자동차 탐지 실행
    stream_and_detect_cars(cctv_url)
    
else:
    print(f"'{search_keyword}'에 해당하는 CCTV를 찾을 수 없습니다.")


'서울'이(가) 포함된 CCTV 목록:
1. CCTV명: [중부선] 동서울영업소 - URL: http://cctvsec.ktict.co.kr/20/P3g2G+dVPybCllrJ8Mhxg9tAmvfDyShcyzKa9N2Am1/G2X22OrDUBS/AnHa+zKqVhYDE+UZ8HPt7T9WUo0hMJSX/gD66wFRZ+HM4xdKc22Q=
2. CCTV명: [경부선] 추풍령휴게소(서울) - URL: http://cctvsec.ktict.co.kr/304/5gpPYXw80U/HFdvymPicataBemgqfUpjuFPRXI/GvbQbs+yhIh9jwiksUO8567ARSmKGbDGJXvRD/QCEi0pneKtiCVvdQADreUW1Vm6/9nU=
3. CCTV명: [경부선] 검단 졸음쉼터(서울) - URL: http://cctvsec.ktict.co.kr/2385/ySyQe/8ShUBdmayeomcdS5Fc/xKQbmrY/14gBKOC3rqEnOrqrmI9hA3NriRKjRYZ/CipCcR0/0QJYT88v/XVwRvhbbq9zobsvjuSXkq4R9E=
4. CCTV명: [경부선] 추풍령쉼터(서울) - URL: http://cctvsec.ktict.co.kr/2399/wbHNzudl6OXzaY3o6LKzwWHqOC+YlR4cNhMW90FA7p1Y8v5YDnifWxbZ+vzENSGpqHevk5bm+joTNrvjHMOfA6XeAlEyWPb6kOKcJIaHw5A=
5. CCTV명: [서울양양선] 미사대교 - URL: http://cctvsec.ktict.co.kr/2569/qHcbJlkQ2AAm0xS0tQ4MAqu5CG5gJr6TutpkYvpRhoOhBPJ6uUYvLI545Vub8Ls15cT6Nn0ysaGcZSV01UfXT1SelBIjWOeOcTHZlXtLxmI=
6. CCTV명: [경부선] 청성쉼터(서울) - URL: http://cctvsec.ktict.co.kr/2735/5tFQ0bvVZRQfyv0EetOJskDlwfej9h0QDnDyspxokekBFbpIKl

Using cache found in C:\Users\SANGMIN/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-12-18 Python-3.10.13 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


YOLOv5 모델이 로드되었습니다.
CCTV 스트림이 시작되었습니다. 종료하려면 'q'를 누르세요.


In [2]:
import requests
import cv2
import torch
import threading
import queue
import time
import logging
from datetime import datetime
import pandas as pd

def detect_cars_yolov5(frame, model, confidence_threshold=0.5):
    """
    YOLOv5 모델을 사용하여 자동차 탐지
    """
    # 모델로 프레임 분석
    results = model(frame)
    detections = results.pandas().xyxy[0]
    
    # 신뢰도 및 클래스 필터링
    detections = detections[
        (detections['confidence'] > confidence_threshold) & 
        (detections['name'].isin(['car', 'truck']))
    ]
    
    return detections

class CarDetectionLogger:
    """
    자동차 탐지 결과 로깅 클래스
    """
    def __init__(self, log_file='car_detections.csv'):
        self.log_file = log_file
        self.log_data = []
        logging.basicConfig(
            filename='car_detection.log', 
            level=logging.INFO, 
            format='%(asctime)s - %(message)s'
        )

    def log_detection(self, frame, detections):
        """
        탐지 결과 로깅
        """
        timestamp = datetime.now()
        car_count = len(detections)
        
        logging.info(f"Detected {car_count} cars")
        
        for _, row in detections.iterrows():
            self.log_data.append({
                'timestamp': timestamp,
                'car_count': car_count,
                'confidence': row['confidence'],
                'bbox': f"({row['xmin']},{row['ymin']},{row['xmax']},{row['ymax']})"
            })

    def save_to_csv(self):
        """CSV 파일로 로그 저장"""
        if self.log_data:
            df = pd.DataFrame(self.log_data)
            df.to_csv(self.log_file, index=False)
            print(f"탐지 로그가 {self.log_file}에 저장되었습니다.")

def initialize_model(device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    YOLOv5 모델 초기화
    """
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', device=device)
    return model

class VideoStreamProcessor:
    """
    비디오 스트림 처리 클래스
    """
    def __init__(self, cctv_url, model, confidence_threshold=0.5):
        self.cctv_url = cctv_url
        self.model = model
        self.confidence_threshold = confidence_threshold
        
        # 큐 초기화
        self.frame_queue = queue.Queue(maxsize=128)
        self.result_queue = queue.Queue(maxsize=128)
        
        # 로거 초기화
        self.logger = CarDetectionLogger()
        
        # 스레드 플래그
        self.is_running = False

    def frame_reader(self):
        """
        프레임 읽기 스레드
        """
        cap = cv2.VideoCapture(self.cctv_url)
        
        while self.is_running:
            ret, frame = cap.read()
            if not ret:
                break
            
            try:
                # 큐가 가득 차지 않도록 대기
                self.frame_queue.put(frame, timeout=2)
            except queue.Full:
                # 큐가 가득 차면 가장 오래된 프레임 제거
                try:
                    self.frame_queue.get_nowait()
                except queue.Empty:
                    pass
        
        cap.release()
        self.is_running = False

    def car_detector(self):
        """
        자동차 탐지 스레드
        """
        while self.is_running:
            try:
                # 프레임 가져오기
                frame = self.frame_queue.get(timeout=2)
                
                # 자동차 탐지
                detections = detect_cars_yolov5(frame, self.model, self.confidence_threshold)
                
                try:
                    # 결과 큐에 추가
                    self.result_queue.put((frame, detections), timeout=2)
                except queue.Full:
                    # 큐가 가득 차면 가장 오래된 결과 제거
                    try:
                        self.result_queue.get_nowait()
                    except queue.Empty:
                        pass
                
                self.frame_queue.task_done()
            
            except queue.Empty:
                # 타임아웃 시 계속 진행
                continue

    def display_results(self):
        """
        결과 디스플레이 스레드
        """
        while self.is_running:
            try:
                # 결과 가져오기
                frame, detections = self.result_queue.get(timeout=2)
                
                # 로깅
                self.logger.log_detection(frame, detections)

                # 탐지된 자동차 표시
                for _, row in detections.iterrows():
                    x1, y1 = int(row['xmin']), int(row['ymin'])
                    x2, y2 = int(row['xmax']), int(row['ymax'])
                    confidence = row['confidence']
                    
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(
                        frame, 
                        f"Car: {confidence:.2f}", 
                        (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        (0, 255, 0), 
                        2
                    )

                # 자동차 수 출력
                car_count = len(detections)
                cv2.putText(
                    frame, 
                    f"Car count: {car_count}", 
                    (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    1, 
                    (0, 0, 255), 
                    2
                )

                # 화면에 프레임 출력
                cv2.imshow("CCTV Stream", frame)
                
                # 결과 큐 작업 완료
                self.result_queue.task_done()

                # 종료 키 확인
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    self.is_running = False
            
            except queue.Empty:
                # 타임아웃 시 계속 진행
                continue

    def start(self):
        """
        스트림 처리 시작
        """
        self.is_running = True

        # 스레드 생성
        reader_thread = threading.Thread(target=self.frame_reader)
        detector_thread = threading.Thread(target=self.car_detector)
        display_thread = threading.Thread(target=self.display_results)

        # 스레드 시작
        reader_thread.start()
        detector_thread.start()
        display_thread.start()

        # 스레드 종료 대기
        reader_thread.join()
        detector_thread.join()
        display_thread.join()

        # 리소스 정리
        cv2.destroyAllWindows()
        self.logger.save_to_csv()

# 나머지 코드 (get_all_cctv_urls, search_cctv_by_name 등)은 이전과 동일

def main():
    # 특정 지역 위도와 경도
    lat = 36.58629
    lng = 128.186793

    # YOLOv5 모델 초기화
    model = initialize_model()

    # 검색할 CCTV 이름 키워드
    search_keyword = input("검색할 CCTV 이름 키워드를 입력하세요: ")

    # 이름으로 CCTV 검색
    result = search_cctv_by_name(lat, lng, search_keyword)

    # 결과 출력 및 스트리밍 선택
    if result:
        print(f"'{search_keyword}'이(가) 포함된 CCTV 목록:")
        for i, cctv in enumerate(result):
            print(f"{i+1}. CCTV명: {cctv['cctvname']} - URL: {cctv['cctvurl']}")
        
        # 사용자로부터 재생할 CCTV 선택
        while True:
            try:
                choice = int(input(f"재생할 CCTV를 선택하세요 (1~{len(result)}): ")) - 1
                if 0 <= choice < len(result):
                    cctv_url = result[choice]['cctvurl']
                    break
                else:
                    print("잘못된 번호입니다. 다시 선택하세요.")
            except ValueError:
                print("숫자를 입력하세요.")

        # 비디오 스트림 프로세서 초기화 및 시작
        processor = VideoStreamProcessor(cctv_url, model)
        processor.start()
    else:
        print(f"'{search_keyword}'에 해당하는 CCTV를 찾을 수 없습니다.")

if __name__ == "__main__":
    main()

Using cache found in C:\Users\SANGMIN/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-12-18 Python-3.10.13 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


NameError: name 'search_cctv_by_name' is not defined